In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 9.7 MB/s 


In [ ]:
!pip install transformers=='4.2.0'


     |████████████████████████████████| 1.8 MB 25.3 MB/s 
     |████████████████████████████████| 2.9 MB 43.5 MB/s 
     |████████████████████████████████| 895 kB 57.6 MB/s 


In [ ]:
# import numpy as np # used for handling numbers
# import pandas as pd # used for handling the dataset
# from sklearn.impute import SimpleImputer # used for handling missing data
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder # used for encoding categorical data
# from sklearn.model_selection import train_test_split # used for splitting training and testing data
# from sklearn.preprocessing import StandardScaler # used for feature scaling

In [ ]:
import pandas as pd
train=pd.read_csv('/content/drive/MyDrive/ACL_sharedtask/Depression/train_final.tsv', sep="\t")

In [ ]:
# train=train.iloc[:,1:]
train

,PID,Text_data,Label
0,train_pid_1,Waiting for my mind to have a breakdown once t...,moderate
1,train_pid_2,My new years resolution : I'm gonna get my ass...,moderate
2,train_pid_3,New year : Somone else Feeling like 2020 will ...,moderate
3,train_pid_4,"My story I guess : Hi, Im from Germany and my ...",moderate
4,train_pid_5,Sat in the dark and cried myself going into th...,moderate
...,...,...,...
13382,dev_pid_4492,Aren’t we all just tired? : I’ve been depresse...,severe
13383,dev_pid_4493,NEED HELP COPING : I had my life pretty much f...,severe
13384,dev_pid_4494,Qutting Zoloft Cold Turkey : I was on 75 mg se...,severe
13385,dev_pid_4495,Crying : I’m coming off my antidepressants and...,severe


In [ ]:
train.rename(columns = {'Text_data':'text'}, inplace = True)

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
train['text'] = train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
train["text"] = train['text'].str.replace('[^\w\s]','')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


In [ ]:
train = train.sample(frac = 1)

# **Load the desired model you want to use**

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModel
  
# tokenizer = AutoTokenizer.from_pretrained("nghuyong/ernie-2.0-en")

# model = AutoModelForSequenceClassification.from_pretrained("nghuyong/ernie-2.0-en",output_hidden_states=True,num_labels=7)

tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-uncased')


model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
train.Label.unique()

array(['moderate', 'severe', 'not depression'], dtype=object)

In [ ]:
labels={'not depression': 0,
 'moderate': 1,
 'severe': 2}
train['Label']= train['Label'].map(labels)


In [ ]:
len(train.Label.unique())


3

In [ ]:
sentences = train.text.values
labels = train.Label.values

In [ ]:
import tqdm as tqdm
import torch

In [ ]:
res = max(train['text'], key = len)
len(res)
res = sum(map(len, train['text']))/float(len(train['text']))
c=0
for i in train['text']:
    if len(i)<350:
        c=c+1
c

7303

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in tqdm.tqdm(sentences):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length =350 ,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation=True
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

  0%|          | 0/13387 [00:00<?, ?it/s]


NameError: ignored

In [ ]:
labels.unique()

NameError: ignored

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation "split.

# Calculate the number of samples to include in each set.
train_size = int(len(dataset))
# val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
# train_dataset = random_split(dataset, [train_size])
print('{:>5,} training samples'.format(train_size))
# print('{:>5,} validation samples'.format(val_size))

22,740 training samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 8
# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            dataset,  # The training samples.
            # sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

In [ ]:
from transformers import AdamW
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 2
# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [ ]:
import datetime
torch.cuda.empty_cache()

In [ ]:
model.cuda()

RuntimeError: ignored

In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = '/content/drive/MyDrive/trial/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

Saving model to /content/drive/MyDrive/trial/


In [ ]:
import random
import numpy as np
import time 
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0
    total_eval_accuracy=0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 1000 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = time.time() - t0
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
         # The call to `model` always returns a tuple, so we need to pull the
        # loss value out of the tuple.
        loss = outputs[0]
            # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    avg_train_accuracy = total_eval_accuracy / len(train_dataloader)
    print(avg_train_loss)
    print(avg_train_accuracy)
    # Measure how long this epoch took.
    training_time = time.time() - t0

    print("")
    # print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    # print("")
    # print("Running Validation...")

    # t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    # model.eval()

    # # Tracking variables 
    # total_eval_accuracy = 0
    # total_eval_loss = 0
    # nb_eval_steps = 0

    # Evaluate data for one epoch
    # for batch in validation_dataloader:
        
    #     # Unpack this training batch from our dataloader. 
    #     #
    #     # As we unpack the batch, we'll also copy each tensor to the GPU using 
    #     # the `to` method.
    #     #
    #     # `batch` contains three pytorch tensors:
    #     #   [0]: input ids 
    #     #   [1]: attention masks
    #     #   [2]: labels 
    #     b_input_ids = batch[0].to(device)
    #     b_input_mask = batch[1].to(device)
    #     b_labels = batch[2].to(device)
        
    #     # Tell pytorch not to bother with constructing the compute graph during
    #     # the forward pass, since this is only needed for backprop (training).
    #     with torch.no_grad():        

    #         # Forward pass, calculate logit predictions.
    #         # token_type_ids is the same as the "segment ids", which 
    #         # differentiates sentence 1 and 2 in 2-sentence tasks.
    #         # The documentation for this `model` function is here: 
    #         # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
    #         # Get the "logits" output by the model. The "logits" are the output
    #         # values prior to applying an activation function like the softmax.
    #         (loss, logits) = model(b_input_ids, 
    #                                token_type_ids=None, 
    #                                attention_mask=b_input_mask,
    #                                labels=b_labels)
            
    #     # Accumulate the validation loss.
    #     # total_eval_loss += loss.item()

    #     # Move logits and labels to CPU
    #     logits = logits.detach().cpu().numpy()
    #     label_ids = b_labels.to('cpu').numpy()

    #     # Calculate the accuracy for this batch of test sentences, and
    #     # accumulate it over all batches.
    #     total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # # Report the final accuracy for this validation run.
    # avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    # print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # # Calculate the average loss over all of the batches.
    # avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # # Measure how long the validation run took.
    # validation_time = format_time(time.time() - t0)
    
    # print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    # print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            # 'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_train_accuracy,
            # 'Training Time': training_time,
            # 'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

# print("Total training took {:} (h:mm:ss)".format(time.time()-total_t0))


======== Epoch 1 / 1 ========
Training...
  Batch 1,000  of  83,716.    Elapsed: 219.45699071884155.
  Batch 2,000  of  83,716.    Elapsed: 438.93046736717224.
  Batch 3,000  of  83,716.    Elapsed: 658.2038781642914.
  Batch 4,000  of  83,716.    Elapsed: 877.3880929946899.
  Batch 5,000  of  83,716.    Elapsed: 1096.4623062610626.
  Batch 6,000  of  83,716.    Elapsed: 1315.6476457118988.
  Batch 7,000  of  83,716.    Elapsed: 1534.8472030162811.
  Batch 8,000  of  83,716.    Elapsed: 1754.1061747074127.
  Batch 9,000  of  83,716.    Elapsed: 1973.5230295658112.
  Batch 10,000  of  83,716.    Elapsed: 2192.819258213043.
  Batch 11,000  of  83,716.    Elapsed: 2412.0858891010284.
  Batch 12,000  of  83,716.    Elapsed: 2631.2872908115387.
  Batch 13,000  of  83,716.    Elapsed: 2850.538642168045.
  Batch 14,000  of  83,716.    Elapsed: 3069.8378920555115.
  Batch 15,000  of  83,716.    Elapsed: 3288.943605661392.
  Batch 16,000  of  83,716.    Elapsed: 3508.0463654994965.
  Batch 17,

# **Load Test and predict labels**

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/ACL_sharedtask/Depression/test_data.tsv',sep="\t")

In [ ]:
test.rename(columns = {'text data':'text'}, inplace = True)
test

,Pid,text
0,test_pid_1,Im scared : This is it. I lie to myself every ...
1,test_pid_2,New to this but just wanted to vent : I just f...
2,test_pid_3,I’m sad : It’s kinda always been an issue. I w...
3,test_pid_4,Lonely but not alone. : All of my immediately ...
4,test_pid_5,This year has been trash. : I dont know why I’...
...,...,...
3240,test_pid_3241,"Feeling lonely. : Hi reddit, I haven’t posted ..."
3241,test_pid_3242,When would suicide be right? : So I got back f...
3242,test_pid_3243,Lowest I’ve ever been ever. : To make a long s...
3243,test_pid_3244,Does the Toxoplasma Gondii ruined my life ? (f...


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
test['text'] = test['text'].apply(lambda y: ' '.join([word for word in y.split() if word not in (stop_words)]))
test["text"] = test['text'].str.replace('[^\w\s]','')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


In [ ]:
# test=pd.read_csv('/content/test_master_multi.csv')
# test.dropna(inplace=True)
sentences2 = test.text.values

input_ids = []
attention_masks = []

for sent in tqdm.tqdm(sentences2):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 350,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

  0%|          | 0/3245 [00:00<?, ?it/s]


NameError: ignored

In [ ]:
batch_size = 64

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# test["text"]= test["text"].astype(str)

In [ ]:
model.eval()

# Tracking variables 
predictions= []

# Predict 
for batch in tqdm.tqdm(prediction_dataloader):
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  
  # Store predictions and true labels
  predictions.append(logits)

print('    DONE.')

100%|██████████| 7/7 [01:15<00:00, 10.73s/it]

    DONE.


In [ ]:
from numpy import exp
def softmax(vector):
	e = exp(vector)
	return e / e.sum()

In [ ]:
res=[]
for val in predictions:
  for m in val:
    temp=softmax(m)
    # print(temp)
    u=np.argmax(temp)
    res.append(u)

In [ ]:
r=pd.DataFrame(res)

In [ ]:
labels={0: 'not depression',
 1: 'moderate',
 2: 'severe'}
test['Labels']= test['Labels'].map(labels)

# **Evaluation of the Model**

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test.status,r[0],digits=4))

              precision    recall  f1-score   support

           0     0.8991    0.7972    0.8451      5755
           1     0.9959    0.9982    0.9970    281272

    accuracy                         0.9941    287027
   macro avg     0.9475    0.8977    0.9211    287027
weighted avg     0.9939    0.9941    0.9940    287027



In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test.Label,r[0])

# **Downnload the csv file of the labels generated**

In [ ]:
test["Labels"]=res

In [ ]:
test.to_csv('Depression_BERT.csv', index=False)